Used tutorial https://github.com/maobedkova/TopicModelling_PySpark_SparkNLP/blob/master/Topic_Modelling_with_PySpark_and_Spark_NLP.ipynb for reference

In [3]:
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from pyspark.sql import types as T

spark = sparknlp.start()

24/11/18 20:48:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [27]:
og_data = spark.read.csv("../data_topicmodel.csv", header= True).select(["id", "cleaned_text"])

In [28]:
#Remove sample
og_data.show(5)
sample_data = og_data.sample(0.001)

+-----+--------------------+
|   id|        cleaned_text|
+-----+--------------------+
|hk5r2|i had an appointm...|
|iqimz|i created this si...|
|pfzt5|hello everyone  i...|
|pk714|i grew up with bo...|
|q0q8x|i have to ask whe...|
+-----+--------------------+
only showing top 5 rows



## Preprocessing

In [29]:
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

In [30]:
tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

In [31]:
normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') 

In [57]:
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot', "selfimprovement", "_"]

stopwords = english + time + reddit + topic_specific

from sparknlp.annotator import StopWordsCleaner

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['normalized']) \
     .setOutputCol('unigrams') \
     .setStopWords(stopwords)

In [33]:
#from sparknlp.annotator import NGramGenerator

#ngrammer = NGramGenerator() \
 #   .setInputCols(['normalized']) \
  #  .setOutputCol('ngrams') \
   # .setN(3) \
    #.setEnableCumulative(True) \
    #.setDelimiter('_')

In [34]:
pos = PerceptronModel.load("/project/macs40123/spark-jars/pos_anc_en_3.0.0_3.0_1614962126490/")\
      .setInputCols("document", "unigrams")\
      .setOutputCol("pos")

In [35]:
from sparknlp.base import Finisher

finisher = Finisher().setInputCols(['unigrams', 'pos'])

In [36]:
my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          stopwords_cleaner,
          #ngrammer,
          pos,
          finisher
      ])

In [37]:
pipelineModel = my_pipeline.fit(sample_data)
processed_data = pipelineModel.transform(sample_data)
processed_data.show(1)

+------+--------------------+--------------------+--------------------+
|    id|        cleaned_text|   finished_unigrams|        finished_pos|
+------+--------------------+--------------------+--------------------+
|1wyudh|i love technology...|[technology, cert...|[NN, JJ, NN, JJ, ...|
+------+--------------------+--------------------+--------------------+
only showing top 1 row



In [38]:
processed_data.columns

['id', 'cleaned_text', 'finished_unigrams', 'finished_pos']

In [39]:
processed_data.select('finished_unigrams').show(10, truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                   finished_unigrams|
+----------------------------------------------------------------------------------------------------+
|[technology, certain, extent, okay, macbook, pro, gaming, pc, xbox, iphone, staples, selling, tec...|
|[giving, away, hours, coaching, three, done, times, always, welcoming, process, move, pardon, shr...|
|[chronically, fantasize, times, least, problematic, creates, great, disparity, disappointment, cu...|
|[fucking, pissed, conduct, hs, interested, classes, given, weeks, school, left, school, pissing, ...|
|[last, found, ex, found, boyfriend, one, hardest, weeks, together, years, child, together, slowly...|
|[title, pretty, explanatory, apologize, throwaway, sure, protocol, deleted, personally, social, p...|
|[recently, kids, made, crude, economic, version, behavioral, marble, jar

In [40]:
processed_data.show(3)

+------+--------------------+--------------------+--------------------+
|    id|        cleaned_text|   finished_unigrams|        finished_pos|
+------+--------------------+--------------------+--------------------+
|1wyudh|i love technology...|[technology, cert...|[NN, JJ, NN, JJ, ...|
|3apk27|hi reddit   im gi...|[giving, away, ho...|[VBG, RB, NNS, VB...|
|6t1o0t|i chronically fan...|[chronically, fan...|[RB, JJ, NNS, JJS...|
+------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [41]:
def filter_unigrams(finished_unigrams, finished_pos):
    '''Filters individual words based on their POS tag'''
    return [word for word, pos in zip(finished_unigrams, finished_pos)
            if pos in ['JJ', 'NN', 'NNS', 'NNPS']]

udf_filter_unigrams = F.udf(filter_unigrams, T.ArrayType(T.StringType()))

In [42]:
processed_data = processed_data.withColumn('filtered_unigrams_by_pos', udf_filter_unigrams(
                                                   F.col('finished_unigrams'),
                                                   F.col('finished_pos')))

processed_data.select("filtered_unigrams_by_pos").show(10, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                            filtered_unigrams_by_pos|
+----------------------------------------------------------------------------------------------------+
|[technology, certain, extent, okay, macbook, pc, xbox, iphone, staples, tech, fun, hand, hand, ti...|
|[hours, times, process, move, pardon, offer, previous, offers, promise, twohour, session, powerfu...|
|[fantasize, times, problematic, creates, great, disparity, disappointment, current, state, depres...|
|[pissed, conduct, hs, interested, classes, weeks, school, school, tf, school, home, phone, watch,...|
|[last, ex, weeks, years, child, realization, entire, years, asocial, slob, relationship, read, ti...|
|[title, explanatory, throwaway, sure, protocol, social, pressures, powerful, forces, proud, ackno...|
|[kids, crude, economic, version, behavioral, marble, jar, classrooms, be

In [44]:
processed_data.columns

['id',
 'cleaned_text',
 'finished_unigrams',
 'finished_pos',
 'filtered_unigrams_by_pos']

In [58]:
#Now that POS was done, lemmatization makes more sense at this point

#Merge tokens as just one string to be able to take it as a document in the new Pipeline
from pyspark.sql import functions as F
tokens_as_string = F.udf(lambda x: ' '.join(x), T.StringType())
processed_data = processed_data.withColumn('joined_tokens', tokens_as_string(F.col('filtered_unigrams_by_pos')))

last_documentAssembler = DocumentAssembler() \
     .setInputCol('joined_tokens') \
     .setOutputCol('joined_document')

last_tokenizer = Tokenizer() \
     .setInputCols(['joined_document']) \
     .setOutputCol('tokenized')
     
lemmatizer = LemmatizerModel.load("../models/lemma_ewt_en_3.4.3_3.0_1651416655397/")\
      .setInputCols("tokenized")\
      .setOutputCol("lemmatized")

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('final') \
     .setStopWords(stopwords)

last_finisher = Finisher() \
     .setInputCols(['final']) \

last_pipeline = Pipeline() \
     .setStages([last_documentAssembler,                  
                 last_tokenizer,
                 lemmatizer,
                 stopwords_cleaner,
                 last_finisher])

final_data = last_pipeline.fit(processed_data).transform(processed_data)


In [59]:
final_data.show(10, truncate=10)
                                                                                                                          

+------+------------+-----------------+------------+------------------------+-------------+--------------+
|    id|cleaned_text|finished_unigrams|finished_pos|filtered_unigrams_by_pos|joined_tokens|finished_final|
+------+------------+-----------------+------------+------------------------+-------------+--------------+
|1wyudh|  i love ...|       [techno...|  [NN, JJ...|              [techno...|   technol...|    [techno...|
|3apk27|  hi redd...|       [giving...|  [VBG, R...|              [hours,...|   hours t...|    [proces...|
|6t1o0t|  i chron...|       [chroni...|  [RB, JJ...|              [fantas...|   fantasi...|    [fantas...|
|8eikjz|  hey rse...|       [fuckin...|  [VBG, J...|              [pissed...|   pissed ...|    [pissed...|
|8j4p29|  last we...|       [last, ...|  [JJ, VB...|              [last, ...|   last ex...|    [last, ...|
|96w9bd|  title i...|       [title,...|  [NN, RB...|              [title,...|   title e...|    [title,...|
|9gwrhq|  recentl...|       [recent..

In [60]:
final_data.select('finished_final').show(50, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                      finished_final|
+----------------------------------------------------------------------------------------------------+
|[technology, certain, extent, okay, macbook, pc, xbox, iphone, staple, tech, fun, hand, hand, cat...|
|[process, move, pardon, offer, previous, offer, promise, twohour, session, powerful, transformati...|
|[fantasize, problematic, create, great, disparity, disappointment, current, state, depressive, li...|
|[pissed, conduct, hs, interested, class, school, school, tf, school, home, phone, watch, youtube,...|
|[last, ex, child, realization, entire, asocial, slob, relationship, read, pain, entire, view, soc...|
|[title, explanatory, throwaway, sure, protocol, social, pressure, powerful, force, proud, acknowl...|
|[kid, crude, economic, version, behavioral, marble, jar, classrooms, beh

In [61]:
final_data.columns

['id',
 'cleaned_text',
 'finished_unigrams',
 'finished_pos',
 'filtered_unigrams_by_pos',
 'joined_tokens',
 'finished_final']

## Vectorization

In [62]:
#Apply TF-IDF filtering
tfizer = CountVectorizer(inputCol='finished_final', outputCol='tf_features', minDF=0.01, maxDF=0.80)
tf_model = tfizer.fit(final_data)
tf_result = tf_model.transform(final_data)

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

## LDA

In [68]:
num_topics = 5
max_iter = 100

lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features', seed=2503)
lda_model = lda.fit(tfidf_result)

In [69]:
vocab = tf_model.vocabulary

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

In [70]:
num_top_words = 15

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=90)

+-----+------------------------------------------------------------------------------------------+
|topic|                                                                                topicWords|
+-----+------------------------------------------------------------------------------------------+
|    0|[mom, expert, drink, weight, mother, early, cold, long, tired, task, energy, word, talk...|
|    1|[purpose, part, reason, journey, discipline, sex, social, line, adhd, sort, fear, simil...|
|    2|[job, family, friend, ego, confidence, age, career, kid, depression, school, mental, ph...|
|    3|[woman, result, conversation, body, man, give, school, true, sport, environment, test, ...|
|    4|[book, anxiety, feeling, pain, certain, last, future, mind, read, look, free, relations...|
+-----+------------------------------------------------------------------------------------------+



# REMEMBER TO CHANGE DATASET TO COMPLETE ONE


In [66]:
lda_model.logPerplexity(tfidf_result)


6.073087460300905

In [67]:
lda_model.logLikelihood(tfidf_result)

-175183.90217685243

Try with different parameters

In [44]:
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(lda.seed, [2503]) \
    .addGrid(lda.k, [5, 8]) \
    .addGrid(lda.maxIter, [10, 20]) \
    .build()

def evaluate_model(model, data):
    log_likelihood = model.logLikelihood(data)
    perplexity = model.logPerplexity(data)
    return log_likelihood, perplexity

best_perplexity = 100
best_log_likelihood = -99999999999
best_param_map_ll = None
best_param_map_pxty = None

for param_map in paramGrid:
    model = lda.copy(param_map).fit(tfidf_result)
    log_likelihood, perplexity = evaluate_model(model, tfidf_result)
    print(f"Params: {param_map}, Log Likelihood: {log_likelihood}, Perplexity: {perplexity}")
    if perplexity < best_perplexity: 
        best_perplexity = perplexity
        best_param_map_pxty = param_map
    if log_likelihood > best_log_likelihood: 
        best_log_likelihood = log_likelihood
        best_param_map_ll = param_map

print("Best parameters for perplexity")
print(best_param_map_pxty)
print(best_perplexity)
print("------------------------------------------")
print("Best parameters for log likelihood")
print(best_param_map_ll)
print(best_log_likelihood)

Params: {Param(parent='LDA_827c4141dbad', name='seed', doc='random seed.'): 2503, Param(parent='LDA_827c4141dbad', name='k', doc='The number of topics (clusters) to infer. Must be > 1.'): 5, Param(parent='LDA_827c4141dbad', name='maxIter', doc='max number of iterations (>= 0).'): 10}, Log Likelihood: -388229.90755375766, Perplexity: 6.7414869595950835


Params: {Param(parent='LDA_827c4141dbad', name='seed', doc='random seed.'): 2503, Param(parent='LDA_827c4141dbad', name='k', doc='The number of topics (clusters) to infer. Must be > 1.'): 5, Param(parent='LDA_827c4141dbad', name='maxIter', doc='max number of iterations (>= 0).'): 20}, Log Likelihood: -382982.50760265486, Perplexity: 6.650367554176167


Params: {Param(parent='LDA_827c4141dbad', name='seed', doc='random seed.'): 2503, Param(parent='LDA_827c4141dbad', name='k', doc='The number of topics (clusters) to infer. Must be > 1.'): 8, Param(parent='LDA_827c4141dbad', name='maxIter', doc='max number of iterations (>= 0).'): 10}, Log Likelihood: -391892.4418861631, Perplexity: 6.8050856854030926


Params: {Param(parent='LDA_827c4141dbad', name='seed', doc='random seed.'): 2503, Param(parent='LDA_827c4141dbad', name='k', doc='The number of topics (clusters) to infer. Must be > 1.'): 8, Param(parent='LDA_827c4141dbad', name='maxIter', doc='max number of iterations (>= 0).'): 20}, Log Likelihood: -385150.12457071117, Perplexity: 6.6880075227597935
Best parameters for perplexity
{Param(parent='LDA_827c4141dbad', name='seed', doc='random seed.'): 2503, Param(parent='LDA_827c4141dbad', name='k', doc='The number of topics (clusters) to infer. Must be > 1.'): 5, Param(parent='LDA_827c4141dbad', name='maxIter', doc='max number of iterations (>= 0).'): 20}
6.650367554176167
------------------------------------------
Best parameters for log likelihood
{Param(parent='LDA_827c4141dbad', name='seed', doc='random seed.'): 2503, Param(parent='LDA_827c4141dbad', name='k', doc='The number of topics (clusters) to infer. Must be > 1.'): 5, Param(parent='LDA_827c4141dbad', name='maxIter', doc='max n